In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import MinMaxScaler , FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

In [3]:
import pickle

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
maths = pd.read_csv('student-mat.csv' , sep=';')
por = pd.read_csv('student-por.csv' , sep=';')

In [6]:
maths = maths.rename(columns={'absences':'absences_M','G1': 'G1_M', 'G2': 'G2_M', 'G3': 'G3_M'})
por = por.rename(columns={'absences':'absences_P','G1': 'G1_P', 'G2': 'G2_P', 'G3': 'G3_P'})

In [7]:
maths

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences_M,G1_M,G2_M,G3_M
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,3,yes,no,yes,no,yes,yes,yes,no,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,yes,yes,yes,yes,yes,yes,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,yes,no,yes,yes,no,no,4,3,2,1,2,5,4,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,course,other,1,2,2,no,yes,yes,no,yes,yes,no,no,5,5,4,4,5,4,11,9,9,9
391,MS,M,17,U,LE3,T,3,1,services,services,course,mother,2,1,0,no,no,no,no,no,yes,yes,no,2,4,5,3,4,2,3,14,16,16
392,MS,M,21,R,GT3,T,1,1,other,other,course,other,1,1,3,no,no,no,no,no,yes,no,no,5,5,3,3,3,3,3,10,8,7
393,MS,M,18,R,LE3,T,3,2,services,other,course,mother,3,1,0,no,no,no,no,no,yes,yes,no,4,4,1,3,4,5,0,11,12,10


In [8]:
maths.columns

Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc',
       'Walc', 'health', 'absences_M', 'G1_M', 'G2_M', 'G3_M'],
      dtype='object')

In [9]:
drop_columns = ['school','sex','age','address','famsize','Pstatus','Medu','Fedu','Mjob','Fjob','reason','guardian','traveltime','studytime','failures','schoolsup','famsup','paid','activities','nursery']
maths = maths.drop(drop_columns , axis=1)
por = por.drop(drop_columns , axis=1)

In [10]:
maths.head()

,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences_M,G1_M,G2_M,G3_M
0,yes,no,no,4,3,4,1,1,3,6,5,6,6
1,yes,yes,no,5,3,3,1,1,3,4,5,5,6
2,yes,yes,no,4,3,2,2,3,3,10,7,8,10
3,yes,yes,yes,3,2,2,1,1,5,2,15,14,15
4,yes,no,no,4,3,2,1,2,5,4,6,10,10


In [11]:
por.head()

,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences_P,G1_P,G2_P,G3_P
0,yes,no,no,4,3,4,1,1,3,4,0,11,11
1,yes,yes,no,5,3,3,1,1,3,2,9,11,11
2,yes,yes,no,4,3,2,2,3,3,6,12,13,12
3,yes,yes,yes,3,2,2,1,1,5,0,14,14,14
4,yes,no,no,4,3,2,1,2,5,0,11,13,13


In [26]:
por.columns

Index(['higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc',
       'Walc', 'health', 'absences_P', 'G1_P', 'G2_P', 'G3_P'],
      dtype='object')

In [12]:
maths.isna().sum() , por.isna().sum()

(higher        0
 internet      0
 romantic      0
 famrel        0
 freetime      0
 goout         0
 Dalc          0
 Walc          0
 health        0
 absences_M    0
 G1_M          0
 G2_M          0
 G3_M          0
 dtype: int64,
 higher        0
 internet      0
 romantic      0
 famrel        0
 freetime      0
 goout         0
 Dalc          0
 Walc          0
 health        0
 absences_P    0
 G1_P          0
 G2_P          0
 G3_P          0
 dtype: int64)

In [13]:
maths.describe(include='all')

,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences_M,G1_M,G2_M,G3_M
count,395,395,395,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
unique,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,yes,yes,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,375,329,263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,3.944304,3.235443,3.108861,1.481013,2.291139,3.554430,5.708861,10.908861,10.713924,10.415190
std,NaN,NaN,NaN,0.896659,0.998862,1.113278,0.890741,1.287897,1.390303,8.003096,3.319195,3.761505,4.581443
min,NaN,NaN,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,3.000000,0.000000,0.000000
25%,NaN,NaN,NaN,4.000000,3.000000,2.000000,1.000000,1.000000,3.000000,0.000000,8.000000,9.000000,8.000000
50%,NaN,NaN,NaN,4.000000,3.000000,3.000000,1.000000,2.000000,4.000000,4.000000,11.000000,11.000000,11.000000
75%,NaN,NaN,NaN,5.000000,4.000000,4.000000,2.000000,3.000000,5.000000,8.000000,13.000000,13.000000,14.000000


In [14]:
def yes_no(data):
    col = [ i for i in data.columns if data[i].dtype == 'object']
    data[col] = data[col].apply(lambda x: x.map({'yes': 1, 'no': 0}))
    return data


In [15]:
yes_no(maths)
yes_no(por)

,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences_P,G1_P,G2_P,G3_P
0,1,0,0,4,3,4,1,1,3,4,0,11,11
1,1,1,0,5,3,3,1,1,3,2,9,11,11
2,1,1,0,4,3,2,2,3,3,6,12,13,12
3,1,1,1,3,2,2,1,1,5,0,14,14,14
4,1,0,0,4,3,2,1,2,5,0,11,13,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,1,1,0,5,4,2,1,2,5,4,10,11,10
645,1,1,0,4,3,4,1,1,1,4,15,15,16
646,1,0,0,1,1,1,1,1,5,6,11,12,9
647,1,1,0,2,4,5,3,4,2,6,10,10,10


In [16]:
model = Pipeline([
    ('scaler', MinMaxScaler()),
    ('regression', MultiOutputRegressor(LinearRegression()))
])

In [17]:
X = maths.drop(['G1_M','G2_M','G3_M'] , axis=1)
y = maths[['G1_M','G2_M','G3_M']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse


3.9544267318226445

In [19]:
scaler = MinMaxScaler()
scaler.fit(X_train)

MinMaxScaler()

In [20]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [21]:
with open('maths_model.pkl', 'wb') as file:
    pickle.dump(model, file)

print("Mathematics model saved as 'maths_model.pkl'")

Mathematics model saved as 'maths_model.pkl'


In [22]:
# Assuming the Portuguese dataset is similar in structure to the mathematics dataset
X_portuguese = por.drop(['G1_P', 'G2_P', 'G3_P'], axis=1)
y_portuguese = por[['G1_P', 'G2_P', 'G3_P']]

X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(X_portuguese, y_portuguese, test_size=0.33, random_state=42)

model.fit(X_train_p, y_train_p)

# Optionally, evaluate the model as before
y_pred_p = model.predict(X_test_p)
rmse_p = np.sqrt(mean_squared_error(y_test_p, y_pred_p))
print("RMSE for Portuguese dataset:", rmse_p)

RMSE for Portuguese dataset: 2.8559089859769022


In [23]:
with open('portuguese_model.pkl', 'wb') as file:
    pickle.dump(model, file)

print("Portuguese model saved as 'portuguese_model.pkl'")

Portuguese model saved as 'portuguese_model.pkl'
